# Integer Programming with PuLP

In [267]:
try:
  from pulp import *
except:
  !pip install pulp
  from pulp import *

## Knapsack Problem

In [268]:
items = {1: {"weight" : 2, "value" : 1},
         2: {"weight" : 3, "value" : 4},
         3: {"weight" : 5, "value" : 5},
         4: {"weight" : 7, "value" : 3},
         5: {"weight" : 11, "value" : 6}}

capacity = 15

In [269]:
model = LpProblem("knapsack-Problem", LpMaximize)
pack = LpVariable.dict("pack", items.keys(), cat = 'Binary')
model += lpSum(pack[i] * items[i]["value"] for i in items)
model += lpSum(pack[i] * items[i]["weight"] for i in items) <= capacity, "weight constraint"
model

knapsack-Problem:
MAXIMIZE
1*pack_1 + 4*pack_2 + 5*pack_3 + 3*pack_4 + 6*pack_5 + 0
SUBJECT TO
weight_constraint: 2 pack_1 + 3 pack_2 + 5 pack_3 + 7 pack_4 + 11 pack_5 <= 15

VARIABLES
0 <= pack_1 <= 1 Integer
0 <= pack_2 <= 1 Integer
0 <= pack_3 <= 1 Integer
0 <= pack_4 <= 1 Integer
0 <= pack_5 <= 1 Integer

In [270]:
model.solve(PULP_CBC_CMD(msg=0))

1

In [271]:
for i in items:
  if pack[i].varValue > 0.8:
    print(f"Item {i} is packed.")

print(f"Total backpack value is {model.objective.value()}.")

Item 2 is packed.
Item 3 is packed.
Item 4 is packed.
Total backpack value is 12.0.


## A Capital Budgeting Problem

* A company has $250,000 available to invest in new projects
* Projects are characterized by their NPV and their ongoing support requirements
* NPV captures the full economic benefit

* For continued support of these projects the following budgets are available:
    * $75,000 in year 2 
    * $50,000 each in years 3, 4, and 5

* Unused funds in any year cannot be carried over

In [272]:
year_budgets = {"Year 1" : 250, "Year 2" : 750, "Year 3" : 50, "Year 4" : 50, "Year 5" : 50}

projects_dict = {
    1: {"NPV": 141, "Year 1": 75, "Year 2": 25, "Year 3": 20, "Year 4": 15, "Year 5": 10},
    2: {"NPV": 187, "Year 1": 90, "Year 2": 35, "Year 3": 0, "Year 4": 0, "Year 5": 30},
    3: {"NPV": 121, "Year 1": 60, "Year 2": 15, "Year 3": 15, "Year 4": 15, "Year 5": 15},
    4: {"NPV": 83, "Year 1": 30, "Year 2": 20, "Year 3": 10, "Year 4": 5, "Year 5": 5},
    5: {"NPV": 265, "Year 1": 100, "Year 2": 25, "Year 3": 20, "Year 4": 20, "Year 5": 20},
    6: {"NPV": 127, "Year 1": 50, "Year 2": 20, "Year 3": 10, "Year 4": 30, "Year 5": 40}
}

model = LpProblem("capital-budget-Problem", LpMaximize)
invest = LpVariable.dict("invest", projects_dict.keys(), cat = 'Binary')
model += lpSum(invest[i] * projects_dict[i]["NPV"] for i in projects_dict)
for year in year_budgets.keys():
    model += lpSum(invest[i] * projects_dict[i][year] for i in projects_dict) <= year_budgets[year], "budget constraint" + str(year)
print(model)

model.solve(PULP_CBC_CMD(msg=0))

for i in projects_dict:
    print(invest[i].varValue)
#  if invest[i].varValue > 0.8:
#    print(f"Project {i} is launched.")

print(f"Total investment value is {model.objective.value()}.")

capital-budget-Problem:
MAXIMIZE
141*invest_1 + 187*invest_2 + 121*invest_3 + 83*invest_4 + 265*invest_5 + 127*invest_6 + 0
SUBJECT TO
budget_constraintYear_1: 75 invest_1 + 90 invest_2 + 60 invest_3 + 30 invest_4
 + 100 invest_5 + 50 invest_6 <= 250

budget_constraintYear_2: 25 invest_1 + 35 invest_2 + 15 invest_3 + 20 invest_4
 + 25 invest_5 + 20 invest_6 <= 750

budget_constraintYear_3: 20 invest_1 + 15 invest_3 + 10 invest_4 + 20 invest_5
 + 10 invest_6 <= 50

budget_constraintYear_4: 15 invest_1 + 15 invest_3 + 5 invest_4 + 20 invest_5
 + 30 invest_6 <= 50

budget_constraintYear_5: 10 invest_1 + 30 invest_2 + 15 invest_3 + 5 invest_4
 + 20 invest_5 + 40 invest_6 <= 50

VARIABLES
0 <= invest_1 <= 1 Integer
0 <= invest_2 <= 1 Integer
0 <= invest_3 <= 1 Integer
0 <= invest_4 <= 1 Integer
0 <= invest_5 <= 1 Integer
0 <= invest_6 <= 1 Integer

1.0
0.0
0.0
1.0
1.0
0.0
Total investment value is 489.0.


## Fixed Charge Problem

This problem involves deciding whether to set up a production process that incurs a fixed cost. The decision includes a fixed setup cost and a variable production cost. If the production is set up, there will be a production profit, but the setup cost must be paid.

### Example:

A factory must decide which production lines to set up for multiple products, each incurring a fixed setup cost and variable production costs. Each product requires a certain amount of a common resource (e.g., machine hours), and there is a maximum market demand for each product. The factory aims to maximize profit while considering the resource constraint and market demand.

In [273]:
import pulp

# Define the problem
prob = pulp.LpProblem("Fixed_Charge_Problem_with_Resource_and_Sales_Constraint", pulp.LpMaximize)

# Define the decision variables
products = ['A', 'B', 'C']
x = pulp.LpVariable.dicts('x', products, lowBound=0, cat='Continuous')
y = pulp.LpVariable.dicts('y', products, cat='Binary')

# Define the parameters
profits = {'A': 10, 'B': 15, 'C': 20}
setup_costs = {'A': 100, 'B': 150, 'C': 200}
resource_requirements = {'A': 5, 'B': 7, 'C': 3}
max_sales = {'A': 50, 'B': 40, 'C': 60}
total_resource_available = 200

# Objective function
prob += pulp.lpSum([x[i] * profits[i] - y[i] * setup_costs[i] for i in products]), "Total Profit"

# Constraints
for i in products:
    prob += x[i] <= y[i] * max_sales[i], f"Max_Sales_Constraint_{i}"
    prob += x[i] >= 0, f"Non_Negative_Production_{i}"
    prob += y[i] >= 0, f"Non_Negative_Setup_{i}"

# Resource constraint
prob += pulp.lpSum([x[i] * resource_requirements[i] for i in products]) <= total_resource_available, "Resource_Constraint"

# Solve the problem
prob.solve()

# Print the results
print("Status:", pulp.LpStatus[prob.status])
for i in products:
    print(f"Production Units for {i} (x) =", pulp.value(x[i]))
    print(f"Setup Decision for {i} (y) =", pulp.value(y[i]))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/059afa15f42645d7aab167e58e0c8ff6-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/059afa15f42645d7aab167e58e0c8ff6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 43 RHS
At line 54 BOUNDS
At line 58 ENDATA
Problem MODEL has 10 rows, 6 columns and 15 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1032.14 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 4 rows, 6 columns (3 integer (3 of which binary)) and 9 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.1
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -1000 iterations 1
Cbc0038I Solution found of -1000
Cbc0038I Relaxin

## Minimum Lot Size

This problem involves deciding on the production of a product that must meet a minimum production lot size. There is a fixed setup cost and a profit per unit produced. The company will only produce if the minimum lot size is met.

### Example:
A company needs to decide whether to produce multiple new products, each with a minimum production lot size. Each product has a specific setup cost, generates a certain profit per unit, requires a common resource, and has a maximum market demand. The goal is to maximize overall profit while considering the resource constraint and market demand.



In [274]:
import pulp

# Define the problem
prob = pulp.LpProblem("Minimum_Lot_Size_Problem_with_Resource_and_Sales_Constraint", pulp.LpMaximize)

# Define the decision variables
products = ['A', 'B', 'C']
x = pulp.LpVariable.dicts('x', products, lowBound=0, cat='Continuous')
y = pulp.LpVariable.dicts('y', products, cat='Binary')

# Define the parameters
profits = {'A': 10, 'B': 15, 'C': 20}
setup_costs = {'A': 100, 'B': 150, 'C': 200}
resource_requirements = {'A': 5, 'B': 7, 'C': 1}
min_lot_sizes = {'A': 30, 'B': 20, 'C': 40}
max_sales = {'A': 50, 'B': 40, 'C': 60}
total_resource_available = 200

# Objective function
prob += pulp.lpSum([x[i] * profits[i] - y[i] * setup_costs[i] for i in products]), "Total Profit"

# Constraints
for i in products:
    prob += x[i] <= y[i] * max_sales[i], f"Max_Sales_Constraint_{i}"
    prob += x[i] >= y[i] * min_lot_sizes[i], f"Minimum_Lot_Size_Constraint_{i}"

# Resource constraint
prob += pulp.lpSum([x[i] * resource_requirements[i] for i in products]) <= total_resource_available, "Resource_Constraint"

# Solve the problem
prob.solve()

# Print the results
print("Status:", pulp.LpStatus[prob.status])
for i in products:
    print(f"Production Units for {i} (x) =", pulp.value(x[i]))
    print(f"Setup Decision for {i} (y) =", pulp.value(y[i]))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8ab2db8a263a4ea3ac1af1431444a6f1-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/8ab2db8a263a4ea3ac1af1431444a6f1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 40 RHS
At line 48 BOUNDS
At line 52 ENDATA
Problem MODEL has 7 rows, 6 columns and 15 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1225 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 7 rows, 6 columns (3 integer (3 of which binary)) and 15 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.3
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -1000 iterations 2
Cbc0038I Solution found of -1000
Cbc0038I Relaxing c

## Quantity Discount

This problem involves deciding on purchasing quantities under a discount scheme. There are different costs associated with purchasing up to a threshold quantity and exceeding it. The objective is to minimize the total cost while meeting the demand.

### Example:
A company needs to purchase masks. The cost per mask is $10 if fewer than 50 masks are bought. If 50 or more masks are bought, the cost per mask drops to $8. The company wants to maximize the profit when selling a mask for $20. They can store at most 200 masks.

In [275]:
import pulp

# Define the problem
prob = pulp.LpProblem("Quantity_Discount_Problem", pulp.LpMaximize)

# Define the decision variables
x1 = pulp.LpVariable('masks_at_price_c1', lowBound=0, cat='Continuous')
x2 = pulp.LpVariable('masks_at_price_c2', lowBound=0, cat='Continuous')
y = pulp.LpVariable('threshold_decision', cat='Binary')

# Define the parameters
c1 = 10  # profit per unit for x1
c2 = 12   # profit per unit for x2
M = 1000  # a large number to facilitate constraint
k = 50  # threshold for discount
storage = 100

# Objective function
prob += x1 * c1 + x2 * c2, "Total Profit"

# Constraints
prob += x2 <= y * M, "Discount_Purchase_Constraint"
prob += x2 >= y * k, "Minimum_Discount_Constraint"
prob += x1+x2<=storage, "storage constaint"

# Solve the problem
prob.solve()

# Print the results
print("Status:", pulp.LpStatus[prob.status])
print("Units at price c1 (x1) =", pulp.value(x1))
print("Units at price c2 (x2) =", pulp.value(x2))
print("Threshold Decision (y) =", pulp.value(y))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.10.13/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5184241048844ef1944729245915b7d5-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/5184241048844ef1944729245915b7d5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 19 RHS
At line 23 BOUNDS
At line 25 ENDATA
Problem MODEL has 3 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1200 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (1 integer (1 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 1.11022e-16
Cbc0038I Solution found of -1200
Cbc0038I Relaxing continuous gives -1200
Cbc0038I Before mini branch and bound, 1 integers at bound fixed a

## Discrete Pricelevels

Here's how many of each drink you expect to sell per hour, depending on the price:

Drink			    $3.50	$4.00	$4.50

Small Latte		    120	    100	    80

Medium Cappuccino	80	    70	    60
 
Large Iced Coffee	90	    85	    75

Barista Time - It takes different amounts of time to make each drink:

* Small Latte: 1 minute
* Medium Cappuccino: 1.5 minutes
* Large Iced Coffee: 0.75 minutes

During the peak hour, you only have 600 minutes of total barista time available.

In [276]:
sales_dict = {
    "Small Latte": {
        3.50 : 120,
        4.00 : 100,
        4.50 : 80,
        "time": 1
        },
    "Medium Cappuccino": {
        3.50: 80,
        4.00: 70,
        4.50: 60,
        "time": 1.5
    },
    "Large Iced Coffee": {
        3.50: 90,
        4.00: 85,
        4.50: 75,
        "time": 0.75
    }
}

In [277]:
model = LpProblem("coffee_shop",LpMaximize)
priceLevels = [3.5, 4, 4.5]

pricingDecision = {}
for product in sales_dict:
    pricingDecision[product] = LpVariable.dict(product, priceLevels, cat="Binary")

pricingDecision


{'Small Latte': {3.5: Small_Latte_3.5, 4: Small_Latte_4, 4.5: Small_Latte_4.5},
 'Medium Cappuccino': {3.5: Medium_Cappuccino_3.5,
  4: Medium_Cappuccino_4,
  4.5: Medium_Cappuccino_4.5},
 'Large Iced Coffee': {3.5: Large_Iced_Coffee_3.5,
  4: Large_Iced_Coffee_4,
  4.5: Large_Iced_Coffee_4.5}}

In [278]:
model += lpSum(lpSum(j * pricingDecision[i][j] * sales_dict[i][j] for j in pricingDecision[i]) for i in pricingDecision)

print(model)

coffee_shop:
MAXIMIZE
315.0*Large_Iced_Coffee_3.5 + 340*Large_Iced_Coffee_4 + 337.5*Large_Iced_Coffee_4.5 + 280.0*Medium_Cappuccino_3.5 + 280*Medium_Cappuccino_4 + 270.0*Medium_Cappuccino_4.5 + 420.0*Small_Latte_3.5 + 400*Small_Latte_4 + 360.0*Small_Latte_4.5 + 0.0
VARIABLES
0 <= Large_Iced_Coffee_3.5 <= 1 Integer
0 <= Large_Iced_Coffee_4 <= 1 Integer
0 <= Large_Iced_Coffee_4.5 <= 1 Integer
0 <= Medium_Cappuccino_3.5 <= 1 Integer
0 <= Medium_Cappuccino_4 <= 1 Integer
0 <= Medium_Cappuccino_4.5 <= 1 Integer
0 <= Small_Latte_3.5 <= 1 Integer
0 <= Small_Latte_4 <= 1 Integer
0 <= Small_Latte_4.5 <= 1 Integer



In [279]:
model += lpSum([[sales_dict[i]['time'] * pricingDecision[i][j] * sales_dict[i][j] for j in pricingDecision[i]] for i in pricingDecision]) <= 150, "barista_time_constraint"


for i in pricingDecision.keys():
    model += lpSum([pricingDecision[i][j] for j in pricingDecision[i]]) <= 1, "pricelevel "+str(i)


print(model)

coffee_shop:
MAXIMIZE
315.0*Large_Iced_Coffee_3.5 + 340*Large_Iced_Coffee_4 + 337.5*Large_Iced_Coffee_4.5 + 280.0*Medium_Cappuccino_3.5 + 280*Medium_Cappuccino_4 + 270.0*Medium_Cappuccino_4.5 + 420.0*Small_Latte_3.5 + 400*Small_Latte_4 + 360.0*Small_Latte_4.5 + 0.0
SUBJECT TO
barista_time_constraint: 67.5 Large_Iced_Coffee_3.5
 + 63.75 Large_Iced_Coffee_4 + 56.25 Large_Iced_Coffee_4.5
 + 120 Medium_Cappuccino_3.5 + 105 Medium_Cappuccino_4
 + 90 Medium_Cappuccino_4.5 + 120 Small_Latte_3.5 + 100 Small_Latte_4
 + 80 Small_Latte_4.5 <= 150

pricelevel_Small_Latte: Small_Latte_3.5 + Small_Latte_4 + Small_Latte_4.5 <= 1

pricelevel_Medium_Cappuccino: Medium_Cappuccino_3.5 + Medium_Cappuccino_4
 + Medium_Cappuccino_4.5 <= 1

pricelevel_Large_Iced_Coffee: Large_Iced_Coffee_3.5 + Large_Iced_Coffee_4
 + Large_Iced_Coffee_4.5 <= 1

VARIABLES
0 <= Large_Iced_Coffee_3.5 <= 1 Integer
0 <= Large_Iced_Coffee_4 <= 1 Integer
0 <= Large_Iced_Coffee_4.5 <= 1 Integer
0 <= Medium_Cappuccino_3.5 <= 1 Integer

In [280]:
model.solve(PULP_CBC_CMD(msg=0))

1

In [281]:
for i in pricingDecision:
    for j in pricingDecision[i]:
        if pricingDecision[i][j].varValue > 0.8:
            print("Product " + i + " is sold for $" + str(j))
        
print("Total profit is $"+ str(model.objective.value()))

Product Small Latte is sold for $4.5
Product Large Iced Coffee is sold for $4
Total profit is $700.0


In [282]:
model.constraints['barista_time_constraint'].slack

6.25

## Continuous Pricing - Single Product

The Setup: You're organizing a music festival and, for now, are only selling General Admission (GA) tickets. You want to find the single best price for these tickets to maximize your revenue.

Data Insights: You've surveyed 10 potential attendees and have gathered data on their maximum willingness-to-pay (WTP) for a GA ticket.

In [283]:
attendee_wtp_dict = {
    1: 60,
    2: 45,
    3: 75,
    4: 55,
    5: 30,
    6: 65,
    7: 40,
    8: 80,
    9: 50,
    10: 70
}

𝑅𝑒𝑣𝑒𝑛𝑢𝑒_𝑖≤ 𝑃𝑟𝑖𝑐𝑒

𝑅𝑒𝑣𝑒𝑛𝑢𝑒_i≤ 𝑊𝑇𝑃_𝑖  ∗ 𝐵𝑢𝑦_𝑖

𝑅𝑒𝑣𝑒𝑛𝑢𝑒_𝑖≥ 𝑃𝑟𝑖𝑐𝑒  − 𝑀 ∗ (1 − 𝐵𝑢𝑦_𝑖)

In [284]:
model = LpProblem("continuous_pricing", LpMaximize)

price = LpVariable("price",0)
buyDecision = LpVariable.dict("buy",attendee_wtp_dict,cat="Binary")
customerRevenue = LpVariable.dict("revenue",attendee_wtp_dict,0)

M = 1000

In [285]:
model += lpSum(customerRevenue[i] for i in attendee_wtp_dict)

for i in attendee_wtp_dict:
    model += customerRevenue[i] <= price, "priceLimitsRevenue"+str(i)
    model += customerRevenue[i] <= buyDecision[i] * attendee_wtp_dict[i], "WTP_limits_revenue"+str(i)
    model += customerRevenue[i] >= price - M * (1-buyDecision[i]), "get_price_if_bought"+str(i)

In [286]:
model.solve(PULP_CBC_CMD(msg=0))

1

In [287]:
price.varValue

45.0

In [288]:
model.objective.value()

360.0

In [289]:
for i in attendee_wtp_dict:
    print(str(attendee_wtp_dict[i]) + "buyDecision: " + str(buyDecision[i].varValue))

60buyDecision: 1.0
45buyDecision: 1.0
75buyDecision: 1.0
55buyDecision: 1.0
30buyDecision: 0.0
65buyDecision: 1.0
40buyDecision: 0.0
80buyDecision: 1.0
50buyDecision: 1.0
70buyDecision: 1.0


## Multiple Products with IC Constraints

In [290]:
# Data
WTP_data = {
    1: {"Basic": 150, "Premium": 80},
    2: {"Basic": 0, "Premium": 0},
    3: {"Basic": 60, "Premium": 90},
    4: {"Basic": 45, "Premium": 65},
    5: {"Basic": 25, "Premium": 40},
    6: {"Basic": 55, "Premium": 85},
    7: {"Basic": 40, "Premium": 60},
    8: {"Basic": 70, "Premium": 100},
    9: {"Basic": 30, "Premium": 45},
    10: {"Basic": 40, "Premium": 2500},
}
students = list(WTP_data.keys())
access_levels = ["Basic", "Premium", "Free"]  # Include "Free" access level

for i in students:
    WTP_data[i]["Free"] = 0  # WTP for "Free" is 0

# Model
model = pulp.LpProblem("Online_Course_Pricing", pulp.LpMaximize)

# Decision variables
prices = pulp.LpVariable.dicts("Price", access_levels, lowBound=0)
buy = pulp.LpVariable.dicts("Buy", ((i, j) for i in students for j in access_levels), cat='Binary')
revenue = pulp.LpVariable.dicts("Revenue", ((i, j) for i in students for j in access_levels), lowBound=0)
utility = pulp.LpVariable.dicts("Utility", ((i, j) for i in students for j in access_levels))  # No lower bound!

# Objective function
model += pulp.lpSum([revenue[(i, j)] for i in students for j in access_levels])

# Constraints
M = 100000  # Big M value

# Individual Rationality (IR) and Revenue Linearization
for i in students:
    for j in access_levels:
        model += prices[j] <= WTP_data[i][j] + M * (1 - buy[(i, j)])
        model += revenue[(i, j)] <= prices[j]
        model += revenue[(i, j)] <= WTP_data[i][j] * buy[(i, j)]
        model += revenue[(i, j)] >= prices[j] - M * (1 - buy[(i, j)])

# Incentive Compatibility (IC)
for i in students:
    for j in access_levels:
        model += utility[(i, j)] == WTP_data[i][j] - prices[j]
        for k in access_levels:
            if k != j:
                model += M * (1 - buy[(i, j)]) + utility[(i, j)] >= utility[(i, k)] - M * buy[(i, k)] 

# One purchase per student (at most), including "Free"
for i in students:
    model += pulp.lpSum([buy[(i, j)] for j in access_levels]) == 1

# Set the price of "Free" to 0
model += prices["Free"] == 0

# Solve the model
model.solve(PULP_CBC_CMD(msg=0))

# Print the solution
if model.status == pulp.LpStatusOptimal:
    print("Optimal Prices:")
    for j in access_levels:
        print(f"  {j}: ${prices[j].varValue:.2f}")

    print("\nPurchase Decisions:")
    for i in students:
        for j in access_levels:
            if buy[(i, j)].varValue == 1:
                print(f"  Student {i} buys {j}")

    print(f"\nTotal Expected Revenue: ${pulp.value(model.objective):.2f}")
else:
    print("No optimal solution found.")

Optimal Prices:
  Basic: $40.00
  Premium: $2500.00
  Free: $0.00

Purchase Decisions:
  Student 1 buys Basic
  Student 2 buys Free
  Student 3 buys Basic
  Student 4 buys Basic
  Student 5 buys Free
  Student 6 buys Basic
  Student 7 buys Basic
  Student 8 buys Basic
  Student 9 buys Free
  Student 10 buys Premium

Total Expected Revenue: $2740.00
